## Import TFRS

https://www.tensorflow.org/recommenders/examples/quickstart

In [1]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

In [2]:
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

### Read the data

In [3]:
# Ratings data. PrefetchDataset
ratings = tfds.load('movielens/100k-ratings', split="train") 
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2021-12-06 11:55:05.620317: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


<MapDataset shapes: {movie_title: (), user_id: ()}, types: {movie_title: tf.string, user_id: tf.string}>

In [6]:
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")
movies = movies.map(lambda x: x["movie_title"])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [7]:
len(movies)

1682

In [10]:
# Build vocabularies to convert user ids and movie titles into integer indices for embedding layers:
user_ids_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

In [11]:
movie_titles_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

### Define a model

define a TFRS model by inheriting from tfrs.Model and implementing the compute_loss method:

In [12]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.
    def __init__(self, user_model: tf.keras.Model, movie_model: tf.keras.Model, task: tfrs.tasks.Retrieval):
        super().__init__()

        # Set up user and movie representations.
        self.user_model = user_model
        self.movie_model = movie_model

        # Set up a retrieval task.
        self.task = task
    
    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        # Define how the loss is computed.

        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])

        return self.task(user_embeddings, movie_embeddings)

Define the two models and the retrieval task.

In [15]:
# Define user models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])

In [17]:
# Define movie models.
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocabulary_size(), 64)
])

In [19]:
# Define your objectives.
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(movies.batch(128).map(movie_model))
)

## Fit and evaluate it

Create the model, train it, and generate predictions:

In [20]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

In [21]:
# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

Epoch 1/3
25/25 [==============================] - 7s 228ms/step - factorized_top_k/top_1_categorical_accuracy: 6.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0014 - factorized_top_k/top_10_categorical_accuracy: 0.0047 - factorized_top_k/top_50_categorical_accuracy: 0.0447 - factorized_top_k/top_100_categorical_accuracy: 0.1019 - loss: 33067.1424 - regularization_loss: 0.0000e+00 - total_loss: 33067.1424
Epoch 2/3
25/25 [==============================] - 6s 229ms/step - factorized_top_k/top_1_categorical_accuracy: 1.4000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0049 - factorized_top_k/top_10_categorical_accuracy: 0.0141 - factorized_top_k/top_50_categorical_accuracy: 0.1053 - factorized_top_k/top_100_categorical_accuracy: 0.2116 - loss: 31015.2791 - regularization_loss: 0.0000e+00 - total_loss: 31015.2791
Epoch 3/3
25/25 [==============================] - 6s 225ms/step - factorized_top_k/top_1_categorical_accuracy: 3.7000e-04 - factorized_top_k/top_5_categorica

In [22]:
# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

In [23]:
# Get some recommendations.
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Top 3 recommendations for user 42: [b'Rent-a-Kid (1995)' b'Just Cause (1995)'
 b'Land Before Time III: The Time of the Great Giving (1995) (V)']


In [25]:
for i in range(50):
    _, titles = index(np.array([str(i)]))
    print(f"Top 5 recommendations for user {i}: {titles[0, :5]}")
    print("\n")

Top 5 recommendations for user 0: [b'Swept from the Sea (1997)' b'Nightwatch (1997)' b'Nightwatch (1997)'
 b'Little Princess, The (1939)' b'Anna Karenina (1997)']


Top 5 recommendations for user 1: [b'Brother Minister: The Assassination of Malcolm X (1994)'
 b'Doom Generation, The (1995)' b'Theodore Rex (1995)'
 b'Turbo: A Power Rangers Movie (1997)' b'Nadja (1994)']


Top 5 recommendations for user 2: [b'3 Ninjas: High Noon At Mega Mountain (1998)' b'Promesse, La (1996)'
 b'Once Upon a Time... When We Were Colored (1995)'
 b'Shall We Dance? (1996)' b"Antonia's Line (1995)"]


Top 5 recommendations for user 3: [b'Critical Care (1997)' b'Spice World (1997)'
 b'How to Be a Player (1997)' b'U Turn (1997)' b'Prophecy II, The (1998)']


Top 5 recommendations for user 4: [b'Assignment, The (1997)' b'Incognito (1997)'
 b'Blues Brothers 2000 (1998)' b'Wonderland (1997)'
 b'Event Horizon (1997)']


Top 5 recommendations for user 5: [b'Amityville: A New Generation (1993)'
 b'Amityville II: The 